In [21]:
news_texts="""Tesla announced it was venturing into auto insurance in 2019. It promised a better and cheaper insurance experience for electric vehicle (EV) drivers beset by high repair costs and premiums. But nearly four years since it launched, Tesla Insurance has faced significant challenges and questions over its viability. A slew of consumer complaints drew lawsuits and regulatory scrutiny last year, and the brakes appear to have been put on Tesla Insurance’s launch in Europe, originally slated for 2023. One analyst said the EV giant seems to have run into the same problems that other tech firms fall into while trying to enter insurance. But he also points out that increased scrutiny over Tesla Insurance could also be part of the “Elon Musk effect” “When I look at all this stuff around Tesla, and all the noise around it, I think some of it is just that Elon Musk is larger than life. People react to him, and they try to jump on any bad news,’ he said. "It's causing them all kinds of public relations problems," Denninginger said. "They’ll have to work their way out of it," he said of Tesla Insurance. "It's not going to be easy, but it's not the end of the world," he concluded. "There’s nothing new here from an insurance industry perspective," he added. “I’m sure they’re fixing it because the company is incredible when you think about what they�’ve done,’ he said.“I'm sure they're fixing it,” he said, “but in the meantime, it’d be nice if they didn’t do this.” “It’ s going to happen to all insurance companies at some point.’ “ “There's nothing new in the insurance industry from a policyholder perspective,“ Denninger said. � � “You’ have to get used to the idea that insurance companies are going to fail.�"""

In [17]:
import os
import requests
import yfinance as yf
from textblob import TextBlob
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from scipy.special import softmax
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import BartTokenizer, BartForConditionalGeneration

In [22]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
sia=SentimentIntensityAnalyzer()
sia.polarity_scores(news_texts)

{'neg': 0.044, 'neu': 0.876, 'pos': 0.08, 'compound': 0.8151}

In [24]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [25]:
model_name="cardiffnlp/twitter-roberta-base-sentiment"
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForSequenceClassification.from_pretrained(model_name)

c:\Users\utkar\Desktop\DG_liger\Project\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [26]:
encoded=tokenizer(news_texts,return_tensors='pt')
output=model(**encoded)
scores=output[0][0].detach().numpy()
scores=softmax(scores)
print(scores)


[0.40104452 0.4877971  0.11115833]


In [27]:
weights = {
    'neg': -1,
    'neu': 0,
    'pos': 1
}
probabilities = {
    'neg': scores[0],
    'neu': scores[1],
    'pos': scores[2]
}
compound_score = sum(probabilities[label] * weights[label] for label in probabilities)
print(f"Compound score: {compound_score}")

Compound score: -0.2898861840367317


In [13]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def find_url(keyword):
    # Convert dates to the format Google expects (M/D/YYYY)
    start_date = "6/8/2023"
    end_date = "6/8/2023"

    search_query = keyword.replace(" ", "+")
    # Construct the URL with the tbs parameter for date range
    link = f"https://www.google.com/search?q={search_query}&tbs=cdr%3A1%2Ccd_min%3A{start_date[0]}%2F{start_date[2]}%2F{start_date[4:]}%2Ccd_max%3A{start_date[0]}%2F{start_date[2]}%2F{start_date[4:]}&tbm=nws"
    print(link)
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(link, headers=headers)
    webpage = response.content
    soup = BeautifulSoup(webpage, 'html5lib')
    links = []
    for div_tag in soup.find_all('div', class_='Gx5Zad'):
        a_tag = div_tag.find('a')
        if a_tag and 'href' in a_tag.attrs:
            href = a_tag['href']
            if href.startswith('/url?q='):
                url = href.split('/url?q=')[1].split('&sa=')[0]
                links.append(url)
    return links

# Example usage
keyword = "elon musk"
start_date = "2024-06-04"
end_date = "2024-06-04"
urls = find_url(keyword)
for url in urls:
    print(url)


https://www.google.com/search?q=elon+musk&tbs=cdr%3A1%2Ccd_min%3A6%2F8%2F2023%2Ccd_max%3A6%2F8%2F2023&tbm=nws


In [23]:
def find_url(keyword):
    start_date = "6/8/2023"    
    root = "https://www.google.com/"
    search_query = keyword.replace(" ", "+")
    link = f"https://www.google.com/search?q={search_query}&tbs=cdr%3A1%2Ccd_min%3A{start_date[0]}%2F{start_date[2]}%2F{start_date[4:]}%2Ccd_max%3A{start_date[0]}%2F{start_date[2]}%2F{start_date[4:]}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(link, headers=headers)
    webpage = response.content
    soup = BeautifulSoup(webpage, 'html5lib')
    links = []
    # print(soup.find_all('div'))
    # print(link)
    
    for div_tag in soup.find_all('div', class_='Gx5Zad'):
        a_tag = div_tag.find('a')
        if a_tag:
            if 'href' in a_tag.attrs:
                href = a_tag['href']
                if href.startswith('/url?q='):
                    url = href.split('/url?q=')[1].split('&sa=')[0]
                    links.append(url)
    return link

In [24]:
a=find_url("elon musk news")
print(a)

https://www.google.com/search?q=elon+musk+news&tbs=cdr%3A1%2Ccd_min%3A6%2F8%2F2023%2Ccd_max%3A6%2F8%2F2023


In [29]:
from datetime import datetime, timedelta
from exa_py import Exa

exa = Exa(api_key="9fccb5a3-0bff-453b-93bf-dd7e82f7a690")

one_week_ago = (datetime.now() - timedelta(days=7))
date_cutoff = one_week_ago.strftime("%Y-%m-%d")

search_query = "hdfc stock latest news "
search_response = exa.search_and_contents(
    search_query, use_autoprompt=True, start_published_date=date_cutoff,end_published_date=date_cutoff
)

urls = [result.url for result in search_response.results]
print("URLs:")
for url in urls:
    print(url)


# result = exa.search_and_contents(
#   "HDFC stock latest news in a week",
#   type="neural",
#   use_autoprompt=True,
#   num_results=10,
#   text=True
# )

results = search_response.results
result_item = results[0]
print(f"{len(results)} items total, printing the first one:")
print(result_item.text)

URLs:
https://www.rediff.com/business/report/markets-at-fresh-highs-on-buying-in-hdfc-bank-ril/20240614.htm
https://www.rediff.com/business/report/hero-motocorp-jsw-energy-among-7-midcap-stocks-likely-to-get-largecap-tag/20240614.htm
https://www.indiaretailing.com/2024/06/14/stanley-lifestyles-eyes-rs-537-crore-via-ipo-issue-to-open-on-june-21/
https://www.indiaretailing.com/2024/06/14/dollar-industries-sets-sights-on-southern-expansion-with-50-new-outlets/
https://www.rediff.com/business/report/exports-up-9-in-may-trade-deficit-widens-to-7-month-high/20240614.htm
https://twitter.com/Least_ordinary/status/1801577486075236861
https://www.rediff.com/business/report/nse-chief-cautions-retail-investors-against-derivatives-trading/20240614.htm
https://x.com/apoorv/status/1801524945983574180
https://www.indiaretailing.com/2024/06/14/retail-tracker-may-store-openings/
https://twitter.com/nimeshscnbc/status/1801448106560733403
10 items total, printing the first one:
Benchmark equity indices Se

In [45]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote

def find_news_url(keyword, start_date, end_date):
    root = "https://www.google.com/"
    search_query = keyword.replace(" ", "+")
    link = f"{root}search?q={search_query}&tbm=nws&tbs=cdr:1,cd_min:{start_date},cd_max:{end_date}"
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    news_links = []
    
    for article in soup.select('div.SoaBEf'):
        link = article.select_one('a')
        if link and 'href' in link.attrs:
            url = link['href']
            if url.startswith('/url?q='):
                url = unquote(url.split('/url?q=')[1].split('&sa=')[0])
            news_links.append(url)
    
    return news_links

# Example usage
keyword = "elon musk"
start_date = "6/8/2023"
end_date = "6/8/2023"
results = find_news_url(keyword, start_date, end_date)
print(len(results))

10


In [61]:
from bs4 import BeautifulSoup
import requests
import yfinance as yf


In [62]:
def scraping_article(url):
    headers = {
    'User-Agent': 'Your User Agent String',
    }
    r=requests.get(url,headers=headers)
    soup=BeautifulSoup(r.text,'html.parser')
    paragraphs=soup.find_all('p')
    text= [paragraph.text for paragraph in paragraphs]
    words=' '.join(text).split(' ')
    article = ' '.join(words)
    return article

In [63]:
url="https://www.motor1.com/news/702440/tesla-plaid-vs-dodge-demon-170-drag-race/"
a=scraping_article(url)

In [64]:
print(a)

In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import spacy

In [16]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import spacy
import re

# Load spaCy for text processing
nlp = spacy.load("en_core_web_sm")

# Load RoBERTa sentiment analysis model
sentiment_analyzer = pipeline("sentiment-analysis", model="roberta-base")

def analyze_keyword_sentiment(text, keyword):
    # Normalize text and keyword
    text = text.lower()
    keyword = keyword.lower()

    # Check if keyword is in the text
    if keyword not in text:
        return "Neutral (Keyword not found)", 0

    # Tokenize the text
    doc = nlp(text)
    
    # Find sentences containing the keyword
    relevant_sentences = [sent.text for sent in doc.sents if keyword in sent.text.lower()]
    
    if not relevant_sentences:
        return "Neutral (Keyword not in complete sentence)", 0

    # Analyze sentiment of relevant sentences
    sentiments = [sentiment_analyzer(sent)[0] for sent in relevant_sentences]
    
    # Calculate weighted sentiment score
    total_score = sum(sentiment['score'] if sentiment['label'] == 'POSITIVE' else -sentiment['score'] for sentiment in sentiments)
    avg_score = total_score / len(sentiments)
    
    # Determine overall sentiment
    if avg_score > 0.1:
        label = "Positive"
    elif avg_score < -0.1:
        label = "Negative"
    else:
        label = "Neutral"

    return label, avg_score

# Example usage

keyword = "byd"

sentiment, score = analyze_keyword_sentiment(news_texts, keyword)
print(f"Sentiment for '{keyword}': {sentiment}")
print(f"Sentiment score: {score:.2f}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sentiment for 'byd': Negative
Sentiment score: -0.53
